In [1]:
import numpy as np

from pathlib import Path
pathtohere = Path.cwd()
print(pathtohere)

C:\Users\02dba\Documents\GitHub\Diatomic-Molecule-Dissociation-by-Gravitational-Gradient\contents


In [2]:
def formatExpressions(motionEquations):
    """
    
    
    """
    
    patterns = {
        'q' : 'p->charge',
        'sigma(r,theta)': 'sigma_p',
        'delta(r)': 'delta_p',
        'chi(theta)': 'chi_p',
        'Power(Q,2)': 'BH->charge2',
        'Power(l,2)': 'BH->l2',
        'Power(a,2) + Power(l,2) + Power(r,2)': 'ral_sqr',
        'Q': 'BH->charge',
        'l': 'BH->l',
        'a': 'BH->a',
        'M': 'BH->mass',
        'Power(a,2)': 'BH->a2',
        'Power(l,2)': 'BH->l2',
        'Power(Q,2)': 'BH->charge2',
        'Power(l,3)': 'BH->l3',
        'Power(a,3)': 'BH->a3',
        'Power(a,4)': 'BH->a4',
        'Power(l,4)': 'BH->l4',
        'Power(a,5)': 'BH->a5',
        'Power(l,5)': 'BH->l5',
        'Power(a,6)': 'BH->a6',
        'Power(l,6)': 'BH->l6',
        'Power(a,7)': 'BH->a7',
        'Power(l,7)': 'BH->l7',
        'Power(a,8)': 'BH->a8',
        'Power(l,8)': 'BH->l8',
        'Power(a,9)': 'BH->a9',
        'Power(l,9)': 'BH->l9',
        'Power(a,10)': 'BH->a10',
        'Cos': 'cos',
        'Sin': 'sin',
        'Csc': '1. / sin',
        'Cot': '1. / tan',
        'Sec': '1. / cos',
        'm' : 'p->mass',
        'r': 'p->r',
        'phi': 'p->phi',
        'theta': 'p->theta',
        'Power(q,2)': '(p->charge*ps->charge)',
        'angular' : 'p->L',
        'energy' : 'p->energy',
        'Power(r,2)': 'p->r2',
        'Power(r,3)': 'p->r3',
        'Power(r,4)': 'p->r4',
        'Power(r,5)': 'p->r5',
        'Power(r,6)': 'p->r6',
        'Power(r,7)': 'p->r7',
        'rs': 'ps->r',
        'phis': 'ps->phi',
        'thetas': 'ps->theta',
        'Power(rdot,2)': '(rdot*rdot)',
        'Power(phidot,2)': '(phidot*phidot)',
        'Power(thetadot,2)': '(thetadot*thetadot)',
        
        'Power(sigma(r,theta),2)' : 'sigma_p2',
        'Power(sigma(r,theta),3)' : 'sigma_p3',
        'Power(sigma(r,theta),4)' : 'sigma_p4',
        'Power(delta(r),2)' : 'delta_p2',
        'Power(delta(r),3)' : 'delta_p3',
        'Power(delta(r),4)' : 'delta_p4',
        'Power(chi(theta),2)' : 'chi_p2',
        'Power(chi(theta),3)' : 'chi_p3',
        'Power(chi(theta),4)' : 'chi_p4',
        
        'Cos(theta)': 'costheta',
        'Sin(theta)': 'sintheta',
        'Sin(2.*theta)': 'sin2theta',
        'Power(Sin(theta),2)': 'sintheta2',
        'Power(Sin(theta),3)': 'sintheta3',
        'Power(Sin(theta),4)': 'sintheta4',
        'Power(Sin(theta),5)': 'sintheta5',
        'Power(Sin(theta),6)': 'sintheta6',
        'Power(Sin(theta),7)': 'sintheta7',
        'Sin(phi)': 'sinphi',
        'Cos(phi)': 'cosphi',
        
        # Source stuff.
        'Sin(thetas)': 'sinthetas',
        'Cos(thetas)': 'costhetas',
        'Sin(phis)': 'sinphis',
        'Cos(phis)': 'cosphis',
        'Sin(phisdot)': 'sinphisdot',
        'Cos(phisdot)': 'cosphisdot',
        'Sin(thetasdot)': 'sinthetasdot',
        'Cos(thetasdot)': 'costhetasdot',
        'Power(Sin(thetasdot),2)': 'sinthetas2',
        'Power(Cos(thetasdot),2)': 'costhetas2',
        'Power(Sin(phisdot),2)': 'sinphis2',
        'Power(Cos(phisdot),2)': 'cosphis2',
        'Power(rsdot,2)': 'rsdot2',
        'Power(phisdot,2)': 'phisdot2',
        
        'epsilon': 'ps->epsilon',
        'sigma0': 'ps->sigma',
        'Power(sigma0,6)': 'ps->sigma6',
        'Power(sigma0,12)': 'ps->sigma12',
        
        # n vector
        'Power(nmag,2)': 'nmag2',
        'Power(nmag,5)': 'nmag5',
        'Power(nmag,6)': 'nmag6',
        'Power(nmag,7)': 'nmag7',
        'Power(nmag,8)': 'nmag8',
        'Power(nmag,12)': 'nmag12',
        'Power(nmag,13)': 'nmag13',
        'Power(nmag,14)': 'nmag14',
        'Power(nx,2)': '(nx*nx)',
        'Power(ny,2)': '(ny*ny)',
        'Power(nz,2)': '(nz*nz)',
        # Keep the same.
        '1.*' : '',
        '2.' : '2.',
        '/2' : '/2.',
        'Power' : 'pow',
        'epsilon0' : 'epsilon_0',
        'nmag' : 'nmag',
        'tdot': 'tdot',
        'rdot' : 'rdot',
        'phidot' : 'phidot',
        'thetadot' : 'thetadot',
        'tsdot': 'tsdot',
        'rsdot' : 'rsdot',
        'phisdot' : 'phisdot',
        'thetasdot' : 'thetasdot',
        'tsddot':'tsddot',
        'rsddot' : 'rsddot',
        'phisddot':'phisddot',
        'thetasddot' : 'thetasddot'
    }
    
    maxPatternLength = max(map(len, patterns.keys()))
    
    for i in range(len(motionEquations)):
        j = 0
        while j < len(motionEquations[i]):
            for k in range(0,maxPatternLength):
                section = motionEquations[i][j:j+maxPatternLength - k] 
                #print(section)
                
                if section in patterns.keys():
                    motionEquations[i] = (motionEquations[i][:j] + patterns[section] 
                                        + motionEquations[i][j+maxPatternLength - k:])
                    j += len(patterns[section]) - 1
                    break
                    
            j += 1
                
                
                    
    return motionEquations



def configureFile():
    """
    Configure the cpp file to find the new coordinates from the Euler-Lagrange equations.
    """
    
    configureIdentifier = '// Configured==true\n'
    
    motionEquations = []
    
    line = True
    filePath = pathtohere / 'src/eulerLagrange.cpp'
    i = 0

    # https://stackoverflow.com/questions/3277503/how-to-read-a-file-line-by-line-into-a-list
    with open(filePath, 'r', encoding='UTF-8') as f:
        while line and i < 4:
            line = f.readline()
            
            if line == configureIdentifier:
                print(f'File already configured.')
                return
            if i==0:
                print(f'File not configured: {configureIdentifier[:-1]} not found.')
            
            
            cut = slice(0,-1) if i<3 else slice(0,None)
            motionEquations.append(line[cut])
            
            i+=1
            
    motionEquations = formatExpressions(motionEquations)
    
    with open(filePath, 'w', encoding='UTF-8') as f:
        f.write(f'{configureIdentifier}')
        f.write("#include \"eulerLagrange.h\"\n\n")

        f.write('const cpp_dec_float_25 epsilon_0{ 1. / (4.*M_PI) }; // Geometrised-Gaussian units\n\n\n')
        
        f.write('std::tuple<cpp_dec_float_25, cpp_dec_float_25,')
        f.write('\tcpp_dec_float_25, cpp_dec_float_25> eulerMoveMathematica(const BlackHole* BH,\n')
        f.write('\tParticle* p, Particle* ps,\n')
        f.write('\tconst cpp_dec_float_25 sigma_p, const cpp_dec_float_25 delta_p,')
        f.write('\tconst cpp_dec_float_25 chi_p, const cpp_dec_float_25 dlambda)\n')
        f.write('{\n')
        f.write('\tcpp_dec_float_25 t_new{};\n')
        f.write('\tcpp_dec_float_25 r_new{};\n')
        f.write('\tcpp_dec_float_25 phi_new{};\n')
        f.write('\tcpp_dec_float_25 theta_new{};\n\n')
        
        f.write('\tconst cpp_dec_float_25 sigma_p2{sigma_p * sigma_p};\n')
        f.write('\tconst cpp_dec_float_25 sigma_p3{sigma_p * sigma_p2};\n')
        f.write('\tconst cpp_dec_float_25 sigma_p4{sigma_p2 * sigma_p2};\n')
        f.write('\tconst cpp_dec_float_25 delta_p2{delta_p * delta_p};\n')
        f.write('\tconst cpp_dec_float_25 delta_p3{delta_p * delta_p2};\n')
        f.write('\tconst cpp_dec_float_25 delta_p4{delta_p2 * delta_p2};\n')
        f.write('\tconst cpp_dec_float_25 chi_p2{chi_p * chi_p};\n')
        f.write('\tconst cpp_dec_float_25 chi_p3{chi_p * chi_p2};\n')
        f.write('\tconst cpp_dec_float_25 chi_p4{chi_p2 * chi_p2};\n\n')
        
        f.write('\tconst cpp_dec_float_25 tdot{ (p->t - p->t_prev) / dlambda };\n')
        f.write('\tconst cpp_dec_float_25 rdot{ (p->r - p->r_prev) / dlambda };\n')
        f.write('\tconst cpp_dec_float_25 phidot{ (p->phi - p->phi_prev) / dlambda };\n')
        f.write('\tconst cpp_dec_float_25 thetadot{ (p->theta - p->theta_prev) / dlambda };\n\n')
        f.write('\tconst cpp_dec_float_25 tsdot{ (ps->t - ps->t_prev) / dlambda };\n')
        f.write('\tconst cpp_dec_float_25 rsdot{ (ps->r - ps->r_prev) / dlambda };\n')
        f.write('\tconst cpp_dec_float_25 phisdot{ (ps->phi - ps->phi_prev) / dlambda };\n')
        f.write('\tconst cpp_dec_float_25 thetasdot{ (ps->theta - ps->theta_prev) / dlambda };\n')
        f.write('\tconst cpp_dec_float_25 tsddot{(ps->t - 2.*ps->t_prev + ps->t_prevprev) / (dlambda*dlambda)};\n')
        f.write('\tconst cpp_dec_float_25 rsddot{(ps->r - 2.*ps->r_prev + ps->r_prevprev) / (dlambda*dlambda)};\n')
        f.write('\tconst cpp_dec_float_25 phisddot{(ps->phi - 2.*ps->phi_prev + ps->phi_prevprev) / (dlambda*dlambda)};\n')
        f.write('\tconst cpp_dec_float_25 thetasddot{')
        f.write('\t(ps->theta - 2.*ps->theta_prev + ps->theta_prevprev) / (dlambda*dlambda)};\n\n')
        
        f.write('\tconst cpp_dec_float_25 nx{p->r*sin(p->theta)*cos(p->phi) - ps->r*sin(ps->theta)*cos(ps->phi)};\n') 
        f.write('\tconst cpp_dec_float_25 ny{p->r*sin(p->theta)*sin(p->phi) - ps->r*sin(ps->theta)*sin(ps->phi)};\n') 
        f.write('\tconst cpp_dec_float_25 nz{p->r*cos(p->theta) - ps->r*cos(ps->theta)};\n')
        f.write('\tconst cpp_dec_float_25 nmag{sqrt(nx*nx + ny*ny + nz*nz)};\n')
        f.write('\tconst cpp_dec_float_25 nmag2{nmag*nmag};\n')
        f.write('\tconst cpp_dec_float_25 nmag5{nmag*nmag2*nmag2};\n')
        f.write('\tconst cpp_dec_float_25 nmag6{nmag5*nmag};\n')
        f.write('\tconst cpp_dec_float_25 nmag7{nmag6*nmag};\n')
        f.write('\tconst cpp_dec_float_25 nmag8{nmag6*nmag2};\n')
        f.write('\tconst cpp_dec_float_25 nmag12{nmag6*nmag6};\n')
        f.write('\tconst cpp_dec_float_25 nmag13{nmag6*nmag7};\n')
        f.write('\tconst cpp_dec_float_25 nmag14{nmag7*nmag7};\n\n')
        

        
        
        f.write('\tconst cpp_dec_float_25 ral_sqr{BH->a2 + BH->l2 + p->r2};\n')
        f.write('\tconst cpp_dec_float_25 sintheta{sin(p->theta)};\n')
        f.write('\tconst cpp_dec_float_25 sintheta2{sintheta*sintheta};\n')
        f.write('\tconst cpp_dec_float_25 sintheta3{sintheta2*sintheta};\n')
        f.write('\tconst cpp_dec_float_25 sintheta4{sintheta2*sintheta2};\n')
        f.write('\tconst cpp_dec_float_25 sintheta5{sintheta2*sintheta3};\n')
        f.write('\tconst cpp_dec_float_25 sintheta6{sintheta3*sintheta3};\n')
        f.write('\tconst cpp_dec_float_25 sintheta7{sintheta3*sintheta4};\n\n')

        f.write('\tconst cpp_dec_float_25 costheta{cos(p->theta)};\n')
        f.write('\tconst cpp_dec_float_25 sin2theta{sin(2.*p->theta)};\n')
        f.write('\tconst cpp_dec_float_25 sinphi{sin(p->phi)};\n')
        f.write('\tconst cpp_dec_float_25 cosphi{cos(p->phi)};\n\n')
        
        f.write('\tconst cpp_dec_float_25 sinthetas{sin(ps->theta)};\n')
        f.write('\tconst cpp_dec_float_25 costhetas{cos(ps->theta)};\n')
        f.write('\tconst cpp_dec_float_25 sinphis{sin(ps->phi)};\n')
        f.write('\tconst cpp_dec_float_25 cosphis{cos(ps->phi)};\n')
        f.write('\tconst cpp_dec_float_25 sinthetasdot{sin(thetasdot)};\n')
        f.write('\tconst cpp_dec_float_25 costhetasdot{cos(thetasdot)};\n')
        f.write('\tconst cpp_dec_float_25 sinphisdot{sin(phisdot)};\n')
        f.write('\tconst cpp_dec_float_25 cosphisdot{cos(phisdot)};\n')
        f.write('\tconst cpp_dec_float_25 sinthetas2{sinthetas*sinthetas};\n')
        f.write('\tconst cpp_dec_float_25 costhetas2{costhetas*costhetas};\n')
        f.write('\tconst cpp_dec_float_25 sinphis2{sinphis*sinphis};\n')
        f.write('\tconst cpp_dec_float_25 cosphis2{cosphis*cosphis};\n')
        f.write('\tconst cpp_dec_float_25 rsdot2{rsdot*rsdot};\n')
        f.write('\tconst cpp_dec_float_25 phisdot2{phisdot*phisdot};\n\n')
        

        
        f.write(f'\tt_new = 2. * p->t - p->t_prev + dlambda*dlambda*({motionEquations[0]});\n')
        f.write(f'\tr_new = 2. * p->r - p->r_prev + dlambda*dlambda*({motionEquations[1]});\n')
        f.write(f'\tphi_new = 2. * p->phi - p->phi_prev + dlambda*dlambda*({motionEquations[2]});\n')
        f.write(f'\ttheta_new = 2. * p->theta - p->theta_prev + dlambda*dlambda*({motionEquations[3]});\n\n')
        
        f.write('\treturn {t_new, r_new, phi_new, theta_new};\n')
        f.write('}\n')
        
        
    filePath = pathtohere / 'include/eulerLagrange.h'
    with open(filePath, 'w', encoding='UTF-8') as f:
        f.write('#ifndef EULERLAGRANGE_H\n')
        f.write('#define EULERLAGRANGE_H\n\n')
        
        f.write('#include <iostream>\n')
        f.write('#include <tuple>\n')
        f.write('#define _USE_MATH_DEFINES\n')
        f.write('#include "math.h"\n\n')
        f.write('#include <boost/multiprecision/cpp_dec_float.hpp>\n')
        f.write('using namespace boost::multiprecision;\n')
        f.write('using cpp_dec_float_25 = number<cpp_dec_float<25>>;\n\n')


        f.write('#include "blackHole.h"\n')
        f.write('#include "particle.h"\n\n\n')
        f.write('std::tuple<cpp_dec_float_25, cpp_dec_float_25,')
        f.write('\tcpp_dec_float_25, cpp_dec_float_25> eulerMoveMathematica(const BlackHole* BH,\n')
        f.write('\tParticle* p, Particle* ps,\n')
        f.write('\tconst cpp_dec_float_25 sigma_p, const cpp_dec_float_25 delta_p,')
        f.write('\tconst cpp_dec_float_25 chi_p, const cpp_dec_float_25 dlambda);\n\n')

        f.write('#endif EULERLAGRANGE_H\n')
        
    print('Configuration complete.')
            
                        
            

In [3]:
def main():
    configureFile()

In [4]:
if __name__=='__main__':
    main()

File not configured: // Configured==true not found.
Configuration complete.
